In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import lightgbm as lgb
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# -----------------------------
# 1. 配置与数据加载
# -----------------------------
DATA_PATH = './tube1.csv'  # 替换为实际文件路径
MODES = ['f3', 'f4', 'f5']

# 载入并清洗数据
df = pd.read_csv(DATA_PATH)
features_to_clean = [f'f{i}' for i in range(7, 17)]
for feat in features_to_clean:
    df = df[df[feat] > 0]

# 构建模式组合列
df['mode'] = df[MODES].astype(str).agg('-'.join, axis=1)

# -----------------------------
# 2. 拐点检测 + RUL 标签构建
# -----------------------------
def detect_knee_point(time, signal, window=5):
    # 简易二阶导数峰值检测
    dy = np.gradient(signal)
    ddy = np.gradient(dy)
    idx = np.argmax(np.abs(ddy))
    return time[idx]

records = []
for mode, group in df.groupby('mode'):
    group = group.sort_values('f1').reset_index(drop=True)
    t = group['f1'].values
    f9 = group['f9'].values
    knee_t = detect_knee_point(t, f9)
    end_t = t[-1]
    # 计算 RUL
    group['RUL'] = end_t - group['f1']
    records.append(group)

df_labeled = pd.concat(records).reset_index(drop=True)

# -----------------------------
# 3. 特征工程（ML vs DL）
# -----------------------------
# 公共：模式独热（新参数 sparse_output=False）
encoder = OneHotEncoder(sparse_output=False)
mode_ohe = encoder.fit_transform(df_labeled[['mode']])
mode_cols = encoder.get_feature_names_out(['mode'])
df_modes = pd.DataFrame(mode_ohe, columns=mode_cols, index=df_labeled.index)

# 数值特征：f9 当前值 + 5步 rolling slope
WINDOW = 5
roll_slope = df_labeled['f9'].diff(WINDOW) / WINDOW
df_feat = pd.DataFrame({
    'f9': df_labeled['f9'],
    'f9_slope': roll_slope.fillna(0)
}, index=df_labeled.index)

# 合并特征
X = pd.concat([df_feat, df_modes], axis=1)
y = df_labeled['RUL']

# 归一化
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# 分割
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# -----------------------------
# 4. 传统 ML: LightGBM 训练
# -----------------------------
lgb_train = lgb.Dataset(X_train, label=y_train)
lgb_eval = lgb.Dataset(X_test, label=y_test, reference=lgb_train)

params = {
    'objective': 'regression',
    'metric': ['mae', 'rmse'],
    'learning_rate': 0.1,
    'num_leaves': 31,
    'verbose': -1
}

gbm = lgb.train(
    params,
    lgb_train,
    num_boost_round=200,
    valid_sets=[lgb_train, lgb_eval]
)

# 评估
y_pred_lgb = gbm.predict(X_test, num_iteration=gbm.best_iteration)
print("LightGBM MAE:", mean_absolute_error(y_test, y_pred_lgb))
print("LightGBM RMSE:", np.sqrt(mean_squared_error(y_test, y_pred_lgb)))
print("LightGBM R2:", r2_score(y_test, y_pred_lgb))

# -----------------------------
# 5. 深度学习: LSTM 模型
# -----------------------------
SEQ_LEN = 30  # 序列长度
BATCH_SIZE = 64
EPOCHS = 30

# 构造序列数据集
class RULDataset(Dataset):
    def __init__(self, X, y, seq_len):
        self.X = X
        self.y = y.values
        self.seq_len = seq_len

    def __len__(self):
        return len(self.y) - self.seq_len + 1

    def __getitem__(self, idx):
        x_seq = self.X[idx:idx+self.seq_len]
        y_val = self.y[idx+self.seq_len-1]
        return torch.tensor(x_seq, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32)

# 创建 DataLoader
df_X_train = pd.DataFrame(X_train)
df_y_train = pd.Series(y_train)
df_X_test = pd.DataFrame(X_test)
df_y_test = pd.Series(y_test)
train_dataset = RULDataset(df_X_train.values, df_y_train, SEQ_LEN)
test_dataset = RULDataset(df_X_test.values, df_y_test, SEQ_LEN)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# 定义 LSTM 回归模型
class LSTMRegressor(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=2):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        return self.fc(out).squeeze()

# 初始化模型
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTMRegressor(input_dim=X_train.shape[1]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

# 训练
for epoch in range(EPOCHS):
    model.train()
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        pred = model(xb)
        loss = criterion(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {loss.item():.4f}")

# 测试
model.eval()
all_preds, all_targets = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device)
        preds = model(xb).cpu().numpy()
        all_preds.append(preds)
        all_targets.append(yb.numpy())

y_pred_lstm = np.concatenate(all_preds)
y_true_lstm = np.concatenate(all_targets)
print("LSTM MAE:", mean_absolute_error(y_true_lstm, y_pred_lstm))
print("LSTM RMSE:", np.sqrt(mean_squared_error(y_true_lstm, y_pred_lstm)))
print("LSTM R2:", r2_score(y_true_lstm, y_pred_lstm))

# -----------------------------
# 6. 小结与对比
# -----------------------------
# 可根据输出的 MAE/RMSE/R2 指标比较两种方法的表现，并进行后续调优。


LightGBM MAE: 759286.1125521327
LightGBM RMSE: 1261687.41461134
LightGBM R2: 0.9195887043375472
Epoch 1/30, Loss: 89770462019584.0000
Epoch 2/30, Loss: 71559448363008.0000
Epoch 3/30, Loss: 116234699931648.0000
Epoch 4/30, Loss: 58691365634048.0000
Epoch 5/30, Loss: 66565223481344.0000
Epoch 6/30, Loss: 125182500929536.0000
Epoch 7/30, Loss: 86437357682688.0000
Epoch 8/30, Loss: 67739343388672.0000


KeyboardInterrupt: 

In [1]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# 假设已有 DataFrame df，包括 'mode'、'f9' 等列
# df = pd.read_csv('data.csv')  # 读取示例数据
DATA_PATH = './tube1.csv'  # 替换为实际文件路径
MODES = ['f3', 'f4', 'f5']

# 载入并清洗数据
df = pd.read_csv(DATA_PATH)
features_to_clean = [f'f{i}' for i in range(7, 17)]
for feat in features_to_clean:
    df = df[df[feat] > 0]

# 构建模式组合列
df['mode'] = df[MODES].astype(str).agg('-'.join, axis=1)

cluster_labels = []
for mode, group in df.groupby('mode'):
    X = group[['f9']].values.reshape(-1, 1)
    best_n = 1
    best_score = -1
    best_labels = [0] * len(group)  # 如果只聚为1类，则所有标签为0
    # 尝试 2 类和 3 类
    for n_clusters in [2, 3]:
        if len(X) >= n_clusters:
            kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(X)
            labels = kmeans.labels_
            # 需要至少两个簇才能计算轮廓系数
            if len(set(labels)) > 1:
                score = silhouette_score(X, labels)
            else:
                score = -1
            if score > best_score:
                best_score = score
                best_n = n_clusters
                best_labels = labels
    # 将结果填充回 cluster_labels 列表
    cluster_labels.extend([f"mode{mode}_c{lbl}" for lbl in best_labels])
# 将 sub_mode 列加入 df
df['sub_mode'] = cluster_labels
# 原 mode 可保留或删除，此处后续将使用 sub_mode 做独热编码

df_encoded = pd.get_dummies(df, columns=['sub_mode'])
# 假设 RUL 是要预测的目标变量
features = ['f9', 'f9_slope'] + [col for col in df_encoded.columns if col.startswith('sub_mode_')]
X = df_encoded[features]
y = df_encoded['RUL']



for mode, group in df.groupby('mode'):
    X_mode = group[['f9', 'f9_slope']].values
    pca = PCA(n_components=2)
    coords = pca.fit_transform(X_mode)
    labels = group['sub_mode'].astype('category').cat.codes
    plt.figure(figsize=(6,5))
    sns.scatterplot(x=coords[:,0], y=coords[:,1], hue=group['sub_mode'], palette='tab10')
    plt.title(f'Mode {mode} 的 f9 聚类分布 (PCA 投影)')
    plt.xlabel('PCA 维度 1')
    plt.ylabel('PCA 维度 2')
    plt.legend(title='sub_mode')
    plt.tight_layout()
    plt.savefig(f'mode{mode}_pca_clusters.png')
    plt.close()

# 示例：每个 cluster 的 f9 vs 时间 曲线（平均趋势）
df['time'] = df.groupby('sub_mode').cumcount()  # 假设每个 sub_mode 内时间连续
df_mean = df.groupby(['sub_mode', 'time'])['f9'].mean().reset_index()
plt.figure(figsize=(8,6))
sns.lineplot(data=df_mean, x='time', y='f9', hue='sub_mode', palette='tab10')
plt.title('各 Cluster 平均 f9 随时间的退化趋势')
plt.xlabel('时间')
plt.ylabel('平均 f9')
plt.legend(title='sub_mode')
plt.tight_layout()
plt.savefig('cluster_f9_vs_time.png')
plt.close()


/opt/anaconda3/envs/ML/lib/python3.13/site-packages/sklearn/base.py:1389: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


KeyError: "['f9_slope'] not in index"